In [ ]:
import sys
import os
sys.path.append("../../TychePlot/")
from SpectraPlot import SpectraPlot
from Filereader import fileToNpArray,npArrayToFile
import numpy as np

In [ ]:
def circle_area(radius):
    return radius**2*np.pi

def steradian_by_diameter_and_distance(distance, diameter_hole):
    theta=np.arctan(diameter_hole/(2*distance))
    #A=r**2*Int_0^pi Int_0^2pi sin(theta) d_theta d_phi
    return 2*np.pi*(1-np.cos(theta))

In [ ]:
## Static Lamp parameters

lamp_on_top_file_high_intens='calib/130mm_6s_100rep'
lamp_on_fiber_high_intens='calib/fiber_30s_100rep'
lamp_on_top_file='calib/130mm_400ms_100rep'
lamp_on_fiber='calib/fiber_4s_100rep'
lamp_calib_file='calib/20200701_Recalibration_GigahertzOptics_365-1800nm'

## Sample Holder
lamp_with_fiber_SH_high_intens='calib/SH_EQE_30s_10rep'
lamp_with_fiber_SH='calib/SH_EQE_4s_10rep'
# Output
SH_calib_filename='calib/SH_EQE_calib'

In [ ]:
fiber_diameter=550*10**-6 #m
lamp_hole_diameter=10*10**-3 #m
sphere_hole_diameter=20*10**-3 #m
sphere_lamp_distance=132.5*10**-3 #m


exp_time_lamp_on_top_high_intens=6 #s
exp_time_lamp_on_fiber_high_intens=30 #s
exp_time_SH_high_intens=30 #s
repetitions_lamp_on_top_high_intens=100 #
repetitions_lamp_on_fiber_high_intens=100 #
repetitions_SH_high_intens=10 #
exp_time_lamp_on_top=0.4#s
exp_time_lamp_on_fiber=4 #s
exp_time_SH=4 #s
repetitions_lamp_on_top=100 #s
repetitions_lamp_on_fiber=100 #s
repetitions_SH=10 #s

error_per_channel=0.1 #percent
stiching_wavelength=440*10**-9

In [ ]:
file_format={
                "skiplines":1,
                "fileEnding":".tsv",
                "separator":"\t",
                "commaToPoint":True,
            }
calib_format={
                "skiplines":2,
                "fileEnding":".dat",
                "separator":"\t",
            }
SH_calib_format={
                "preSting":"Wavelengths (m)\tSpectral Radiant Flux (W/nm)",
                "separator":"\t",
                "fileEnding":".clbr",
}

In [ ]:
lamp_on_top_high_intens=fileToNpArray(lamp_on_top_file_high_intens, **file_format)[0]
lamp_on_fiber_high_intens=fileToNpArray(lamp_on_fiber_high_intens, **file_format)[0]
SH_high_intens=fileToNpArray(lamp_with_fiber_SH_high_intens, **file_format)[0]
lamp_on_top=fileToNpArray(lamp_on_top_file, **file_format)[0]
lamp_on_fiber=fileToNpArray(lamp_on_fiber, **file_format)[0]
SH=fileToNpArray(lamp_with_fiber_SH, **file_format)[0]
lamp_calib=fileToNpArray(lamp_calib_file, **calib_format)[0]

In [ ]:
lamp_on_top_counts_per_second=lamp_on_top[:,1]/(exp_time_lamp_on_top*repetitions_lamp_on_top)
lamp_on_fiber_counts_per_second=lamp_on_fiber[:,1]/(exp_time_lamp_on_fiber*repetitions_lamp_on_fiber)
SH_counts_per_second=SH[:,1]/(exp_time_SH*repetitions_SH)
lamp_on_top_high_intens_counts_per_second=lamp_on_top_high_intens[:,1]/(exp_time_lamp_on_top_high_intens*repetitions_lamp_on_top_high_intens)
lamp_on_fiber_high_intens_counts_per_second=lamp_on_fiber_high_intens[:,1]/(exp_time_lamp_on_fiber_high_intens*repetitions_lamp_on_fiber_high_intens)
SH_high_intens_counts_per_second=SH_high_intens[:,1]/(exp_time_SH_high_intens*repetitions_SH_high_intens)
wavelengths=lamp_on_top[:,0]

In [ ]:
lamp_calib[:,0]=lamp_calib[:,0]*10**-9
lamp_calib_func=SpectraPlot.lum_interpolator(lamp_calib.T, kind="cubic")
lamp_calib_W_per_sr_sqm_nm=lamp_calib_func(wavelengths)

In [ ]:
lamp_calib_on_fiber_W_per_nm=lamp_calib_W_per_sr_sqm_nm*2*np.pi*circle_area(fiber_diameter/2)

In [ ]:
lamp_calib_on_top_W_per_nm_sqm=lamp_calib_W_per_sr_sqm_nm*steradian_by_diameter_and_distance(sphere_lamp_distance, sphere_hole_diameter)
lamp_calib_on_top_W_per_nm=lamp_calib_on_top_W_per_nm_sqm*circle_area(lamp_hole_diameter/2)

In [ ]:
# W_per_nm / counts_per_s_nm = J/count
calib_on_fiber_J_per_count=lamp_calib_on_fiber_W_per_nm/lamp_on_fiber_counts_per_second
calib_on_top_J_per_count=lamp_calib_on_top_W_per_nm/lamp_on_top_counts_per_second
calib_on_fiber_high_intens_J_per_count=lamp_calib_on_fiber_W_per_nm/lamp_on_fiber_high_intens_counts_per_second
calib_on_top_high_intens_J_per_count=lamp_calib_on_top_W_per_nm/lamp_on_top_high_intens_counts_per_second

In [ ]:
import scipy.interpolate as sci
import scipy.integrate as scin
import matplotlib.pyplot as plt
plt.plot(wavelengths, calib_on_fiber_J_per_count, wavelengths, calib_on_top_J_per_count, wavelengths, calib_on_fiber_high_intens_J_per_count, wavelengths, calib_on_top_high_intens_J_per_count)
#plt.ylim(10**-,10**3)
plt.yscale("log")

In [ ]:
plt.clf()
plt.plot(wavelengths, lamp_on_fiber_counts_per_second, wavelengths, lamp_on_top_counts_per_second, wavelengths, lamp_on_fiber_high_intens_counts_per_second, wavelengths, lamp_on_top_high_intens_counts_per_second)
#plt.ylim(10**-,10**3)
plt.yscale("log")

In [ ]:
plt.clf()
plt.plot(wavelengths, calib_on_fiber_J_per_count/calib_on_top_J_per_count, wavelengths, calib_on_fiber_high_intens_J_per_count/calib_on_top_high_intens_J_per_count)
#plt.ylim(10**-,10**3)
plt.yscale("log")

In [ ]:
stiching_index=np.argwhere(wavelengths > stiching_wavelength)[0][0]
lowest_valid_calib_index=np.argwhere(wavelengths > lamp_calib[0][0])[0][0]
wavelengths_cropped=wavelengths[lowest_valid_calib_index:]
stiched_calib_on_fiber_J_per_count=np.hstack((calib_on_fiber_high_intens_J_per_count[lowest_valid_calib_index:stiching_index],calib_on_fiber_J_per_count[stiching_index:]))
stiched_calib_on_top_J_per_count=np.hstack((calib_on_top_high_intens_J_per_count[lowest_valid_calib_index:stiching_index],calib_on_top_J_per_count[stiching_index:]))
stiched_lamp_on_fiber_counts_per_second=np.hstack((lamp_on_fiber_high_intens_counts_per_second[lowest_valid_calib_index:stiching_index],lamp_on_fiber_counts_per_second[stiching_index:]))
stiched_lamp_on_top_counts_per_second=np.hstack((lamp_on_top_high_intens_counts_per_second[lowest_valid_calib_index:stiching_index],lamp_on_top_counts_per_second[stiching_index:]))
stiched_SH_counts_per_second=np.hstack((SH_high_intens_counts_per_second[lowest_valid_calib_index:stiching_index],SH_counts_per_second[stiching_index:]))
# calib_spec= C * meas_spec
# calib_spec_on_top= C_KC * lamp_on_top_spec
# calib_spec_on_fiber= C_KFC * lamp_on_fiber_spec
#
# with that:
#
# calib_spec_on_fiber= C_KFC * lamp_on_fiber_spec
# calib_spec_on_fiber= C_F * C_KC * lamp_on_fiber_spec
# calib_spec_on_fiber= C_F * C_KC * lamp_on_fiber_spec
# C_F=calib_spec_on_fiber/(C_KC*lamp_on_fiber_spec)
# fiber_calib_spec=C_KC*lamp_on_fiber_spec
# 
# real_spec= C_SH * C_KC * meas_spec
# C_SH = fiber_calib_spec / (C_KC * meas_spec)

C_F=lamp_calib_on_fiber_W_per_nm[lowest_valid_calib_index:]/(stiched_lamp_on_fiber_counts_per_second*stiched_calib_on_top_J_per_count)
C_KC=stiched_calib_on_top_J_per_count
C_KFC=stiched_calib_on_fiber_J_per_count
calib_spec_after_fiber=C_KC*stiched_lamp_on_fiber_counts_per_second
C_SH=calib_spec_after_fiber/(C_KC*stiched_SH_counts_per_second)

In [ ]:
plt.plot(wavelengths_cropped,stiched_calib_on_fiber_J_per_count , wavelengths_cropped, stiched_calib_on_top_J_per_count)
#plt.ylim(10**-,10**3)
plt.yscale("log")

In [ ]:
plt.plot(wavelengths_cropped,lamp_calib_on_fiber_W_per_nm[lowest_valid_calib_index:], wavelengths_cropped, lamp_calib_on_top_W_per_nm[lowest_valid_calib_index:])
#plt.ylim(10**-,10**3)
plt.yscale("log")

In [ ]:
plt.clf()
plt.plot(wavelengths_cropped, stiched_lamp_on_fiber_counts_per_second, wavelengths_cropped, stiched_lamp_on_top_counts_per_second)
#plt.ylim(10**-,10**3)
plt.yscale("log")

In [ ]:
plt.clf()
#interpolate_calibration
SH_calib_interpol=SpectraPlot.lum_interpolator((wavelengths_cropped,C_SH*C_KC), kind='cubic')
SH_calib_full=SH_calib_interpol(wavelengths)
plt.plot(wavelengths_cropped, C_SH*C_KC, wavelengths, SH_calib_full)
#plt.ylim(10**-,10**3)
plt.yscale("log")

In [ ]:
plt.clf()
plt.plot(wavelengths_cropped, C_KC*stiched_SH_counts_per_second)
#plt.ylim(10**-,10**3)
plt.yscale("log")

In [ ]:
fiber_flux_func=sci.interp1d(wavelengths_cropped,stiched_lamp_on_fiber_counts_per_second*stiched_calib_on_top_J_per_count, kind='cubic')
# Integration
fiber_total_flux=scin.quad(fiber_flux_func,wavelengths_cropped[0],wavelengths_cropped[-1])
fiber_total_flux[0]*10**9

In [ ]:
npArrayToFile(SH_calib_filename,np.vstack((wavelengths,SH_calib_full)).T,**SH_calib_format)

In [ ]:
lamp_calib